# Demo for `OSGenerator.py`
## Import

In [1]:
from osgenerator import OSGenerator
from sys import exit

## Building the graph

In [2]:
osg = OSGenerator()
osg.build("data/friends.txt", "data/checkins.txt")

Building place graph (2/2): 100%|████████████| 6442892/6442892 [01:29<00:00, 72345.76it/s]


## Show information about POI `1946746`
This gives information about latitude, longitude, as well as users checked in at this place

*All `place_id`s are shifted by `1900654`*

In [3]:
osg.get_placeinfo(1946746)

{'placeid': 1946746, 'lat': 51.459313, 'lng': 5.3915300833}

## Find all places visited by user `1000`

In [4]:
osg.get_checkins(1000)

[2071063, 3340957, 1946746, 3335022, 3333213]

## When `1000` visited `1946746`

In [5]:
osg.get_checkintimes(1000, 1946746)

[datetime.datetime(2010, 7, 14, 15, 22, 5),
 datetime.datetime(2010, 7, 13, 9, 46, 22)]

## Find user `1000`'s friends

In [6]:
osg.get_friends(1000)

[1, 207, 615, 1395, 4979, 43588]

## Show OS for user `1000`

In [7]:
os1 = osg.get_object_summary(1000, 1, True)
print(os1)

[USER]
| user_id
|    1000
|
|---[FRIEND]
|    user_id  friend_id
|       1000      43588
|       1000       4979
|       1000        615
|       1000        207
|       1000       1395
|    There are 1 row(s) omitted
|
|---[CHECKIN]
    | user_id  place_id                        time
    |    1000   2071063 16-07-2010 14:47:1279284473
    |    1000   3340957 14-07-2010 20:18:1279131512
    |    1000   1946746 14-07-2010 15:22:1279113725
    |    1000   1946746 13-07-2010 09:46:1279007182
    |    1000   3335022 13-07-2010 16:50:1279032659
    | There are 1 row(s) omitted
    |
    |---[LOCATION]
         place_id  latitude  longitude
          2071063 52.365189   4.936566
          3340957 51.431036   5.463815
          1946746 51.459313   5.391530
          3335022 51.557459   5.090109
          3333213 51.456549   5.395113


## Getting relevant places for user `1000`

Places visited by user `1000` and their friend

In [8]:
# direct_visit = set([d["placeid"] for d in os1["checksin_at"]])
# friend_visit = set()
# friend_visit_list = []
# for friend in os1["friends"]:
#     friend_checkins = osg.get_checkins(friend)
#     for place in friend_checkins:
#         placeid = visit["placeid"]
#         if placeid in direct_visit or placeid in friend_visit:
#             continue
#         friend_visit.add(placeid)
#         v = deepcopy(visit)
#         v.update(time=osg.get_checkintimes(friend, placeid, True))
#         friend_visit_list.append(v)

## Visualizing user `165697`

In [9]:
import numpy as np
import folium

In [10]:
user = np.random.randint(0, 190000)
user = 165697

os_display = osg.get_object_summary(user, 1, False)
os_display_coords = os_display.df_location[["latitude", "longitude"]]
os_display_loc = np.mean(os_display_coords, axis=0)

try:
    with open("data/mapboxkey.txt") as apikey: mapbox_key = next(apikey)
except FileNotFoundError:
    print("Make sure mapboxkey.txt file is present")

tile_url = f"https://api.mapbox.com/styles/v1/misaeljordan/clbbbe7ee001214macmpl7fdj/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_key}"
mapbox_attr = "© <a href='https://www.mapbox.com/about/maps/'>Mapbox</a>"
map_os_display = folium.Map(location=os_display_loc,
                            tiles=tile_url,
                            zoom_start=6,
                            max_zoom=18,
                            control_scale=True,
                            attr=mapbox_attr)

for i, row in os_display_coords.iterrows():
    lat, lng = row["latitude"], row["longitude"]
    folium.CircleMarker(location=[lat, lng], radius=4, color="#d0d0d0").add_to(map_os_display)

In [11]:
map_os_display

## TODO: define objects' weights and similarity function

### Candidate objects' weight definition
$$o.w_{user = A} = \frac{n\ times\ user\ A\ checked\ in\ at\ o}{total\ check\ ins\ by\ user\ A}$$

### Candidate similarity function definition
1. Jaccard Coefficient
$$ Sim(o_1, o_2) = \frac{|o_1.users\cap o_2.users|}{|o_1.users\cup o_2.users|}$$